In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///Users/diegofernandezgil/Library/Caches/pypoetry/artifacts/2f/99/3c/fb06a2044cc764f42d0be784d951aee7812ca33ccd0bbb2585025633c4/scikit_learn-1.1.1-cp39-cp39-macosx_10_13_x86_64.whl


In [4]:
import pickle
import pandas as pd
from mlops_zoomcamp.settings import MODELS_DIR, DATA_DIR

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/Users/diegofernandezgil/Library/Caches/pypoetry/virtualenvs/mlops-zoomcamp-loGnfk92-py3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/diegofernandezgil/Library/Caches/pypoetry/virtualenvs/mlops-zoomcamp-loGnfk92-py3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [10]:
y_pred.mean()

16.191691679979066

In [18]:
year = 2021
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_'  + df.index.astype('str')
df['duration'] = y_pred


In [19]:
df_results = df[['ride_id', 'duration']]

In [20]:
df_results

,ride_id,duration
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


In [21]:
df_results.to_parquet(
    DATA_DIR / "df_predictions.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)